In [3]:
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk import pos_tag
from nltk.chunk import ne_chunk
import string
import re # Importing the 're' module for regular expression operations
import html
import unicodedata
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt_tab')

import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
df = pd.read_csv('/content/drive/MyDrive/AMIT_AI/KAGGLE/DATA_SET/fake-jobs/Fake Postings.csv')

In [5]:
df

,title,description,requirements,company_profile,location,salary_range,employment_type,industry,benefits,fraudulent
0,Mental health nurse,Arm drive court sure vote. Earn $5000/week! Im...,"Basic knowledge in live, no degree required. F...",Rivera and Sons - Established 2022.,West Jeffrey,$55016-$100476,Internship,IT,Free meals,1
1,Conference centre manager,Government whom its bed go tax tree black. Ear...,"Basic knowledge in seek, no degree required. F...","Davidson, Jones and Gomez - Established 2003.",Lake Meredithberg,$53438-$93138,Part-Time,Finance,Flexible hours,1
2,"Engineer, land",I member discuss follow way there nation. Earn...,"Basic knowledge in worker, no degree required....",Allen Ltd - Established 1998.,Lake Cathybury,$45584-$105229,Part-Time,IT,Free travel,1
3,Forest/woodland manager,House across wait approach face. Earn $5000/we...,"Basic knowledge in example, no degree required...",Forbes Ltd - Established 1990.,South Matthewstad,$66188-$139621,Full-Time,Education,Free travel,1
4,"Production designer, theatre/television/film",Case best environmental full finally leader me...,"Basic knowledge in smile, no degree required. ...","Jennings, Martin and Sanchez - Established 1975.",East Rhondafurt,$32183-$115012,Temporary,Retail,Flexible hours,1
...,...,...,...,...,...,...,...,...,...,...
9995,"Designer, furniture",Worry own pressure stuff together room propert...,"Basic knowledge in discussion, no degree requi...",Olson-Williams - Established 2017.,Paulabury,$39450-$149734,Full-Time,Retail,Free meals,1
9996,"Therapist, speech and language",Enter bit thing certainly. Earn $5000/week! Im...,"Basic knowledge in value, no degree required. ...",Moreno-Pruitt - Established 2016.,Moraleschester,$49324-$111597,Part-Time,IT,Sign-on bonus,1
9997,"Therapist, sports",Visit goal under boy. Earn $5000/week! Immedia...,"Basic knowledge in during, no degree required....","Lewis, Patterson and Cowan - Established 1979.",Christinemouth,$41346-$89686,Full-Time,Education,Remote work opportunities,1
9998,Clinical research associate,Walk money letter few. Earn $5000/week! Immedi...,"Basic knowledge in can, no degree required. Fl...",Diaz-Wilkerson - Established 2019.,Lake Meredithberg,$65604-$149614,Contract,Finance,Flexible hours,1


## Check for duplicates

In [6]:
df.duplicated().sum()

0

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            10000 non-null  object
 1   description      10000 non-null  object
 2   requirements     10000 non-null  object
 3   company_profile  10000 non-null  object
 4   location         10000 non-null  object
 5   salary_range     10000 non-null  object
 6   employment_type  10000 non-null  object
 7   industry         10000 non-null  object
 8   benefits         10000 non-null  object
 9   fraudulent       10000 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 781.4+ KB


### Data Preprocessing

In [8]:
df['text'] = df['title'] + ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + df['requirements'] + ' ' + df['benefits']

In [9]:
X = df['text']
y = df['fraudulent']

In [10]:
X[6]

'Presenter, broadcasting Singh Group - Established 1979. Evidence always item increase partner necessary describe. Earn $5000/week! Immediate hiring. Contact now at ramirezronald@yahoo.com. Basic knowledge in interest, no degree required. Flexible hours. Flexible hours'

In [11]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()



def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    # words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [12]:
def nomalize_corpus(corpus):
    return [normalize_text(t) for t in corpus]

In [13]:
X_pre = nomalize_corpus(X)

In [14]:
X[6]

'Presenter, broadcasting Singh Group - Established 1979. Evidence always item increase partner necessary describe. Earn $5000/week! Immediate hiring. Contact now at ramirezronald@yahoo.com. Basic knowledge in interest, no degree required. Flexible hours. Flexible hours'

In [15]:
X_pre[6]

'presenter broadcast singh group establish evidence always item increase partner necessary describe earn week immediate hire contact ramirezronaldyahoocom basic knowledge interest degree require flexible hour flexible hour'

In [18]:
text = X_pre[6]

In [19]:
# Download spaCy's English model (en_core_web_sm)
!python -m spacy download en_core_web_sm

# Step 2: Import necessary libraries
import spacy

# Load spaCy's pre-trained English model
nlp = spacy.load('en_core_web_sm')

# Step 3: Process the text using spaCy's NLP pipeline
doc = nlp(text)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [20]:
# Part of speach tagging
pos_tag = [(token.text, token.pos_) for token in doc]
print(pos_tag)

[('presenter', 'NOUN'), ('broadcast', 'VERB'), ('singh', 'PROPN'), ('group', 'NOUN'), ('establish', 'VERB'), ('evidence', 'NOUN'), ('always', 'ADV'), ('item', 'NOUN'), ('increase', 'NOUN'), ('partner', 'NOUN'), ('necessary', 'ADJ'), ('describe', 'NOUN'), ('earn', 'NOUN'), ('week', 'NOUN'), ('immediate', 'ADJ'), ('hire', 'NOUN'), ('contact', 'NOUN'), ('ramirezronaldyahoocom', 'PROPN'), ('basic', 'ADJ'), ('knowledge', 'NOUN'), ('interest', 'NOUN'), ('degree', 'NOUN'), ('require', 'VERB'), ('flexible', 'ADJ'), ('hour', 'NOUN'), ('flexible', 'ADJ'), ('hour', 'NOUN')]


In [21]:
# NER => Named Entity Recognition
ner = [(ent.text, ent.label_) for ent in doc.ents]
print(ner)

[('singh', 'GPE'), ('week', 'DATE'), ('hour flexible hour', 'TIME')]


In [22]:
# Dependacy Parsing
dep_parse = [(token.text, token.dep_, token.head.text) for token in doc]
print(dep_parse)

[('presenter', 'nsubj', 'broadcast'), ('broadcast', 'compound', 'group'), ('singh', 'compound', 'group'), ('group', 'nsubj', 'establish'), ('establish', 'ROOT', 'establish'), ('evidence', 'dobj', 'establish'), ('always', 'advmod', 'item'), ('item', 'advcl', 'establish'), ('increase', 'compound', 'partner'), ('partner', 'dobj', 'item'), ('necessary', 'amod', 'describe'), ('describe', 'compound', 'week'), ('earn', 'compound', 'week'), ('week', 'npadvmod', 'establish'), ('immediate', 'amod', 'ramirezronaldyahoocom'), ('hire', 'nmod', 'ramirezronaldyahoocom'), ('contact', 'compound', 'ramirezronaldyahoocom'), ('ramirezronaldyahoocom', 'nmod', 'degree'), ('basic', 'amod', 'degree'), ('knowledge', 'compound', 'degree'), ('interest', 'compound', 'degree'), ('degree', 'nsubj', 'require'), ('require', 'conj', 'establish'), ('flexible', 'amod', 'hour'), ('hour', 'dobj', 'require'), ('flexible', 'amod', 'hour'), ('hour', 'npadvmod', 'require')]


In [23]:
# visualize parse tree
from spacy import displacy

displacy.render(doc, style='dep', jupyter=True)